In [32]:
import mne 
import numpy as np
from openpyxl import load_workbook
from glob import glob

In [33]:
#Load raw data subject1
bids_fname = "D:\DATASET PRINCIPALE\sub-01\eeg\sub-01_task-rsvp_eeg.vhdr"
raw= mne.io.read_raw_brainvision(bids_fname, preload=True)

Extracting parameters from D:\DATASET PRINCIPALE\sub-01\eeg\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...


In [35]:
subs_file_path=glob("..\..\DATASET PRINCIPALE\CartellaEEG/*.vhdr")
print(len(subs_file_path))

2


In [65]:
xls_files=glob("..\..\DATASET PRINCIPALE\Cartellaexcel/*.xlsx")
print(len(xls_files))



def prep_data(file_path):
    data= mne.io.read_raw_brainvision(file_path, preload=True)
    return data

2


In [66]:
data_sub_vector= [prep_data(i)for i in subs_file_path]

Extracting parameters from ..\..\DATASET PRINCIPALE\CartellaEEG\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...
Extracting parameters from ..\..\DATASET PRINCIPALE\CartellaEEG\sub-02_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3227279  =      0.000 ...  3227.279 secs...


In [36]:
def get_vectors(data_path):

    book=load_workbook(data_path)
    sheet=book.active
    
    #onset
    onset=[]
    for row in sheet['a2':'a22249']:
        for cel in row:
               onset.append(cel.value)
                
    onset=np.array(onset)
    onset=onset*0.001
    
    #duration 
    duration=np.ones((22248,), dtype=int)*0.01
    
    #description 
    animal=['animal']
    description=[]
    for row in sheet ['e2':'e22249']:
        for cel in row:
            if(cel.value in animal):
                description.append(cel.value)
            else: 
                description.append('E 1')
    return [onset, duration, description] 

In [37]:
matrix=[]

In [38]:
for i in range(len(xls_files)):
    matrix=[get_vectors(i)for i in xls_files]

In [74]:
print(len(matrix[0][2]))
print(len(matrix[1][2]))

22248
22248


In [75]:
def fix_events(data_vector, matrix):
    myannot=[]
    for i in range(len(xls_files)):
            myannot[i]=mne.Annotations(onset=matrix[i][0], duration=matrix[i][1],description=matrix[i][2])
            data_vector[i].set_annotations(myannot[i])
            
    return data_vector

In [76]:
data_vector_note_okay= fix_events(data_sub_vector, matrix)

TypeError: annotations must be an instance of Annotations, got <class 'collections.OrderedDict'> instead

In [59]:
for  i in range(len(xls_files)):
    print(i)

0
1


In [63]:
raw.set_annotations(myaanot)


<RawBrainVision | sub-01_task-rsvp_eeg.eeg, 63 x 3035740 (3035.7 s), ~1.43 GB, data loaded>

In [64]:
#GUARDO GLI EVENTI CHE CI SONO NEL DATASET
events=mne.events_from_annotations(raw)
events

Used Annotations descriptions: ['E 1', 'animal']


(array([[  10724,       0,   10001],
        [  10824,       0,   10001],
        [  10924,       0,   10001],
        ...,
        [3022927,       0,   10001],
        [3023027,       0,   10001],
        [3023127,       0,   10001]]),
 {'E 1': 10001, 'animal': 10002})

In [68]:
events=mne.events_from_annotations(raw)
events
#prima colonna --> sample number 
#terxa colonna--> event number 

#QUANTI SONO
b=len(events[0][events[0][:,2]==10002])
a=len(events[0][events[0][:,2]==10001])



print("Numero di altro:",a)
print("Numero di animali:",b)

e=len(events[0])
print("Numero totale di eventi:",e)


Used Annotations descriptions: ['E 1', 'animal']
Numero di altro: 20952
Numero di animali: 1296
Numero totale di eventi: 22248
